# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Applied Data Science Capstone - Week 3 - Segmenting and Clustering Neighborhoods in Toronto

In [ ]:
import requests
import pandas as pd

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

!pip install geocoder
import geocoder

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: - 

## Question 1: sgrab data and create a clean dataframe

In [87]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# sgrab data from Wikipedia
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]

# clean the data
df = df[df['Borough'] != 'Not assigned']
df.reset_index(inplace=True)
df = df[['Postcode', 'Borough', 'Neighbourhood']]

# merge Neighbourhood with same ['Postcode','Borough']
df['Neighbourhood'] = df.groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x))
df.drop_duplicates(inplace=True)

# dataframe dimension
df.shape

(103, 3)

## Question 2: Add geo location

In [90]:
# load/fetch location files
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.rename(columns= { "Postal Code": "Postcode"}, inplace=True)

# add location columns to the postcode dataframe
toronto_data = pd.merge(df, df_geo[['Postcode', 'Latitude', 'Longitude']], on='Postcode')

## Question 3: Analysis

Define the Borough

In [96]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

toronto_data.dtypes

The dataframe has 11 boroughs and 103 neighborhoods.


Postcode          object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

Draw Toronto map

In [104]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

The geograpical coordinate of Toronto are 43.653963, -79.387207.
